In [101]:
import pandas as pd
import numpy as np
from functools import partial

df = pd.read_excel('node-edges.xlsx', header=None, )
df.columns = df.iloc[61]
df = df.iloc[0:30]

count = 0
for i in range(0, len(df.columns)):
    if (i % 2 != 0):
        df.columns.values[i] = ("%02d" % count) + ': Destination'
        count += 1
        
slices = []
for i in range(2, len(df.columns), 2):
    s = df.iloc[:, i-2:i]
    s = s.dropna()
    s = s[s.iloc[:,1] != '-'] # edges are marked at entrance AND destination, so missing values can be removed
    slices.append(s)
    
slices_dict = {s.iloc[:,0].name : s for s in slices}

def extract_prefix(index, row):
    return " ".join(row[0].split(" ", index)[:index])

def extract_prefix_aqua_hideout(row):
    if 'Warps' in row[0] or 'B1F' in row[0]:
        return extract_prefix(4, row)
    else:
        return extract_prefix(3, row)

def split_sub_nodes(node, extract_prefix):
    sub_nodes = []
    # current_group = []
    current_prefix = ""
    start = 0
    end = 0
    for row in node.iloc:
        name = extract_prefix(row)
        if current_prefix == name:
            end += 1
        else:
            sub_nodes.append(node.iloc[start:end])
            start = end
            end += 1
            current_prefix = name
    sub_nodes.append(node.iloc[start:end])
    # The first iteration adds an empty dataframe. This removes it.
    return sub_nodes[1:]


def process_sub_nodes(key_str, extract_prefix, slices_dict):
    sub_nodes = split_sub_nodes(slices_dict.pop(key_str), extract_prefix)
    key_num, key_name = key_str.split(':')
    
    for i, node in enumerate(sub_nodes):
        # new_key = key_num + '-' + str(i) + ':' + key_name
        new_key = key_str.replace(':', '-%d:' % i)
        node = node.rename({key_str: new_key}, axis='columns')
        slices_dict[new_key] = node


process_sub_nodes('00: Places of Interest', partial(extract_prefix, 2), slices_dict)
process_sub_nodes('12: Granite Cave', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('13: Abandoned Ship', partial(extract_prefix, 3), slices_dict)
process_sub_nodes('15: Meteor Falls', partial(extract_prefix, 3), slices_dict)  
process_sub_nodes('18: Mt Pyre', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('20: Magma Hideout', partial(extract_prefix, 3), slices_dict)  
process_sub_nodes('21: Aqua Hideout', extract_prefix_aqua_hideout, slices_dict)
process_sub_nodes('24: Seafloor Cavern', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('25: Pokecenters', partial(extract_prefix, 2), slices_dict)
process_sub_nodes('26: Victory Road', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('28: E4 Pokecenter', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('31: Sky Pillar', partial(extract_prefix, 3), slices_dict) 

slices = slices_dict.values()

nodes = {}
edges = {}
for s in slices:
    node_name = s.iloc[:,0].name
    node_name = str.strip(node_name.split(':')[1]) # remove number and whitespace from name
    edges.update({e: node_name for e in s.iloc[:,0]})
    nodes.update({node_name: {'node_name': node_name,
                              'transitions': [{'entrance': e, 'destination': d} for e, d in s.iloc],
                              'visited':  False,
                              'previous': {}}})
    
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        
graph = Graph(nodes, edges)

In [69]:
def compare_prefix_row(prefix, row, index):
    name = " ".join(row[0].split(" ", index)[:index])
    return prefix == name

def make_compare_prefix_row(index):
    
    def compare_prefix_row_closure(prefix, row):
        name = " ".join(row[0].split(" ", index)[:index])
        return prefix == name
    
    return compare_prefix_row_closure

def compare_prefix_row_aqua_hideout(prefix, row):
    row_str = row[0]
    if 'Warp' in row[0]:
        return compare_prefix_row(prefix, row, 4)
    else:
        return compare_prefix_row(prefix, row, 3)

In [96]:
from functools import partial

def extract_prefix(index, row):
    return " ".join(row[0].split(" ", index)[:index])

def extract_prefix_aqua_hideout(row):
    if 'Warps' in row[0] or 'B1F' in row[0]:
        return extract_prefix(4, row)
    else:
        return extract_prefix(3, row)
        

In [97]:
def split_sub_nodes(node, extract_prefix):
    sub_nodes = []
    # current_group = []
    current_prefix = ""
    start = 0
    end = 0
    for row in node.iloc:
        # TODO: pass an extractor function instead of the compare?
        name = extract_prefix(row)
        # TODO: pass a compare function for aqua hideout?
        if current_prefix == name:
            end += 1
        else:
            sub_nodes.append(node.iloc[start:end])
            start = end
            end += 1
            current_prefix = name
    sub_nodes.append(node.iloc[start:end])
    # The first iteration adds an empty dataframe. This removes it.
    return sub_nodes[1:]


def process_sub_nodes(key_str, extract_prefix, slices_dict):
    sub_nodes = split_sub_nodes(slices_dict.pop(key_str), extract_prefix)
    key_num, key_name = key_str.split(':')
    
    for i, node in enumerate(sub_nodes):
        # new_key = key_num + '-' + str(i) + ':' + key_name
        new_key = key_str.replace(':', '-%d:' % i)
        node = node.rename({key_str: new_key}, axis='columns')
        slices_dict[new_key] = node

slices_dict_copy = slices_dict.copy()
process_sub_nodes('21: Aqua Hideout', extract_prefix_aqua_hideout, slices_dict_copy)
list(slices_dict_copy.keys())

['00: Places of Interest',
 '01: Oldale/Petalburg',
 '02: Rustboro',
 '03: Dewford',
 '04: Slateport/Mauville/Verdanturf',
 '05: Fallarbor/Lavaridge',
 '06: Fortree/Lilycove',
 '07: Mossdeep',
 '08: Pacifidlog',
 '09: Sootopolis',
 '10: Petalburg Woods',
 '11: Rusturf Tunnel',
 '12: Granite Cave',
 '13: Abandoned Ship',
 '14: Jagged Pass',
 '15: Meteor Falls',
 '16: Mirage Tower',
 '17: Route 119/123',
 '18: Mt Pyre',
 '19: Surf Hub',
 '20: Magma Hideout',
 '22: Lilycove Contest Hall',
 '23: Lilycove Department Store',
 '24: Seafloor Cavern',
 '25: Pokecenters',
 '26: Victory Road',
 '27: Waterfall Hub',
 '28: E4 Pokecenter',
 '29: Navel Rock',
 '30: Dive Hub',
 '31: Sky Pillar',
 '21-0: Aqua Hideout',
 '21-1: Aqua Hideout',
 '21-2: Aqua Hideout',
 '21-3: Aqua Hideout',
 '21-4: Aqua Hideout',
 '21-5: Aqua Hideout',
 '21-6: Aqua Hideout']

In [99]:
slices_dict_copy['21-0: Aqua Hideout']

61,21-0: Aqua Hideout,21: Destination
0,Aqua Hideout B1F NE Room N,Petalburg SW
1,Aqua Hideout B1F NE Room SE,Mossdeep NW
2,Aqua Hideout B1F NE Room SW,Dewford SE


In [67]:
slices_dict['21: Aqua Hideout']

61,21: Aqua Hideout,21: Destination
0,Aqua Hideout B1F NE Room N,Petalburg SW
1,Aqua Hideout B1F NE Room SE,Mossdeep NW
2,Aqua Hideout B1F NE Room SW,Dewford SE
4,Aqua Hideout B1F S Room E,R111 Old Lady
5,Aqua Hideout B1F S Room NW,Oldale Pokecenter
6,Aqua Hideout B1F S Room SW,Department Store 5F Down
8,Aqua Hideout B2F E,Dead-End
9,Aqua Hideout B2F NW,Slateport Contest
10,Aqua Hideout B2F SW,Slateport Mart Interior
12,Aqua Hideout Warps 1 N,Mossdeep Pokecenter W


In [66]:
def process_aqua_hideout():
    pass

In [65]:
for s in slices:
    print(s.iloc[:,0].name)
# print([s.iloc[:,0].name for s in slices])

01: Oldale/Petalburg
02: Rustboro
03: Dewford
04: Slateport/Mauville/Verdanturf
05: Fallarbor/Lavaridge
06: Fortree/Lilycove
07: Mossdeep
08: Pacifidlog
09: Sootopolis
10: Petalburg Woods
11: Rusturf Tunnel
14: Jagged Pass
16: Mirage Tower
17: Route 119/123
19: Surf Hub
21: Aqua Hideout
22: Lilycove Contest Hall
23: Lilycove Department Store
27: Waterfall Hub
29: Navel Rock
30: Dive Hub
00-0: Places of Interest
00-1: Places of Interest
00-2: Places of Interest
00-3: Places of Interest
00-4: Places of Interest
00-5: Places of Interest
00-6: Places of Interest
00-7: Places of Interest
00-8: Places of Interest
00-9: Places of Interest
12-0: Granite Cave
12-1: Granite Cave
12-2: Granite Cave
13-0: Abandoned Ship
13-1: Abandoned Ship
13-2: Abandoned Ship
15-0: Meteor Falls
15-1: Meteor Falls
18-0: Mt Pyre
18-1: Mt Pyre
18-2: Mt Pyre
18-3: Mt Pyre
20-0: Magma Hideout
20-1: Magma Hideout
24-0: Seafloor Cavern
24-1: Seafloor Cavern
24-2: Seafloor Cavern
24-3: Seafloor Cavern
24-4: Seafloor Cav

In [53]:
slices_dict = {s.iloc[:,0].name : s for s in slices}
for i in range(9):
    print(slices_dict['00-%d: Places of Interest' % i])


61 00-0: Places of Interest          00: Destination
0     Rustboro Gym Interior  Pacifidlog Pokecenter W
61 00-1: Places of Interest 00: Destination
2     Mauville Gym Interior   Verdanturf SE
61 00-2: Places of Interest       00: Destination
3    Lavaridge Gym Interior  Seafloor Cavern R3 S
61 00-3: Places of Interest     00: Destination
8                 E4 Sidney  Dewford Pokecenter
61 00-4: Places of Interest      00: Destination
10                E4 Glacia  Lilycove Pokecenter
61 00-5: Places of Interest 00: Destination
11                 E4 Drake     Fortree NNE
61 00-6: Places of Interest         00: Destination
19                  Groudon  Department Store 3F Up
61 00-7: Places of Interest 00: Destination
20                    Ho-oh     Fortree NNW
61 00-8: Places of Interest  00: Destination
21                    Lugia  Mt Pyre 2F Down


In [63]:
slices_dict = {s.iloc[:,0].name : s for s in slices}

# TODO: Can this be generalized for dungeons?
def split_sub_nodes(node, split_at):
    sub_nodes = []
    # current_group = []
    current_prefix = ""
    start = 0
    end = 0
    for row in node.iloc:
        name = " ".join(row[0].split(" ", split_at)[:split_at])
        if current_prefix == name:
            end += 1
        else:
            sub_nodes.append(node.iloc[start:end])
            start = end
            end += 1
            current_prefix = name
    sub_nodes.append(node.iloc[start:end])
    # The first iteration adds an empty dataframe. This removes it.
    return sub_nodes[1:]


def process_sub_nodes(key_str, split_at, slices_dict):
    sub_nodes = split_sub_nodes(slices_dict.pop(key_str), split_at)
    key_num, key_name = key_str.split(':')
    for i, node in enumerate(sub_nodes):
        # new_key = key_num + '-' + str(i) + ':' + key_name
        new_key = key_str.replace(':', '-%d:' % i)
        node = node.rename({key_str: new_key}, axis='columns')
        slices_dict[new_key] = node

        
process_sub_nodes('15: Meteor Falls', 3, slices_dict)
slices_dict['15-1: Meteor Falls']
# list(slices_dict.values())

61,15-1: Meteor Falls,15: Destination
11,Meteor Falls B1F Left SW,Pacifidlog SW
13,Meteor Falls B1F Right N,Surf
14,Meteor Falls B1F Right S,R111 Winstrate House
15,Meteor Falls B1F Right W,Rusturf Tunnel W


In [41]:
import copy

def graph_algo(src, dest, graph):
    if src == dest:
        return True
    src_node = graph.nodes[src]
    src_node['visited'] = True
    queue = [src_node]

    while len(queue) > 0:
        # print('Queue: ', sorted([x['node_name'] for x in queue]))
        current = queue.pop(0)
        # if current['node_name'] == dest:
            # print('Current: %s equals Dest: %s' % (current['node_name'], dest))
            # print('### FOUND IT ####')
            # print(current)
        for t in current['transitions']:
            node_name = graph.edges.get(t['destination'])
            # the node might not exist: Strength, one-way, the 32: Marts thing is not really present in the sheet.
            if node_name:
                node = graph.nodes[node_name]
                if node['visited']:
                    continue
                # add previous
                prev = node['previous']
                if prev.get(current['node_name']):
                    prev[current['node_name']].append(t)
                else:
                    prev[current['node_name']] = [t]
                # mark visited
                # if not node['visited']:
                    # node['visited'] = True
                    # queue.append(node)
                node['visited'] = True
                queue.append(node)
    return graph.nodes[dest]

graph_copy = copy.deepcopy(graph)
dest = graph_algo('Slateport/Mauville/Verdanturf', 'Rustboro', graph_copy)
dest

{'node_name': 'Rustboro',
 'transitions': [{'entrance': 'Rustboro Mart',
   'destination': 'Petalburg Woods SE'},
  {'entrance': 'Rustboro Pokecenter',
   'destination': 'Sootopolis Pokecenter S'},
  {'entrance': 'Rustboro SE', 'destination': 'Meteor Falls 1F E'},
  {'entrance': 'Rustboro Cut House', 'destination': 'Slateport Pokecenter S'}],
 'visited': True,
 'previous': {'Petalburg Woods': [{'entrance': 'Petalburg Woods SE',
    'destination': 'Rustboro Mart'}]}}

In [47]:
def flatten_prev(node_name, graph):
    node = graph.nodes[node_name]
    return [(node[0], trans) for node in list(node['previous'].items())
                        for trans in node[1]]
def list_prev(node_name, graph):
    node = graph.nodes[node_name]
    flat_prev = flatten_prev(node_name, graph)
    return flat_prev
    

print(flatten_prev('Rustboro', graph_copy)[0])
print(flatten_prev('Petalburg Woods', graph_copy)[0])
print(flatten_prev('Seafloor Cavern', graph_copy)[0])
print(flatten_prev('Mt Pyre', graph_copy)[0])
# print(flatten_prev('Slateport/Mauville/Verdanturf', graph_copy)[0])


list_prev('Slateport/Mauville/Verdanturf', graph_copy)

('Petalburg Woods', {'entrance': 'Petalburg Woods SE', 'destination': 'Rustboro Mart'})
('Seafloor Cavern', {'entrance': 'Seafloor Cavern R1 SE', 'destination': 'Petalburg Woods N'})
('Mt Pyre', {'entrance': 'Mt Pyre 2F Up', 'destination': 'Seafloor Cavern R1 N'})
('Slateport/Mauville/Verdanturf', {'entrance': 'Slateport Museum', 'destination': 'Mt Pyre 2F Ceiling 1'})


[]

In [4]:
graph_copy.nodes['Pokecenters']['previous']

NameError: name 'graph_copy' is not defined

In [ ]:
reachable = set()

def reach(node_name, graph, depth_max):
    node = graph.nodes[node_name]
    prev = node['previous'].keys()
    # print(prev)
    for k in prev:
        reachable.add(k)
        if depth_max > 0:
            reach(k, graph, depth_max-1)
reach('Rustboro', graph_copy, 5)
len(reachable)

In [ ]:
name = dest['node_name']
the_node = graph_copy.nodes[name]
prev = the_node['previous']
flat_prev = [[(node[0], trans)] for node in list(prev.items())
    for trans in node[1]]

def list_prev(node_name):
    node = graph_copy.nodes[node_name]
    flat_prev = [(node[0], trans) for node in list(node['previous'].items())
                                    for trans in node[1]]
    
    
flat_prev[0]